In [127]:
import pandas as pd
import numpy as np

In [128]:
DATA_PATH = './data/'

users = pd.read_csv(DATA_PATH + 'users_preprocessed.csv')
books = pd.read_csv(DATA_PATH + 'books_merged.csv')
train = pd.read_csv(DATA_PATH + 'train_ratings.csv')
test = pd.read_csv(DATA_PATH + 'test_ratings.csv')
sub = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [129]:
features_name = ['isbn']

context_df = train.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')
train_df = test.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')
test_df =sub.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')

In [108]:
context_df

,user_id,isbn,rating,age,location_city,location_state,location_country
0,8,0002005018,4,18.0,ontario_others,ontario,canada
1,67544,0002005018,7,30.0,toronto,ontario,canada
2,123629,0002005018,8,30.2,kingston,ontario,canada
3,200273,0002005018,8,18.0,ontario_others,ontario,canada
4,210926,0002005018,9,26.5,guelph,ontario,canada
...,...,...,...,...,...,...,...
306790,278843,0743525493,7,28.0,california_others,california,usa
306791,278851,067161746X,6,33.0,dallas,texas,usa
306792,278851,0884159221,7,33.0,dallas,texas,usa
306793,278851,0912333022,7,33.0,dallas,texas,usa


In [109]:
train['isbn'].nunique()

129777

In [110]:
# 책 이름을 기준으로 평균 평점을 merge
mean_ratings = train.groupby('isbn')['rating'].mean().to_frame()
count_ratings = train.groupby('isbn')['rating'].count().to_frame()

In [111]:
print(len(mean_ratings))
print(len(count_ratings))

129777
129777


In [112]:
isbn_mean_count = mean_ratings.merge(count_ratings, on='isbn', how='left')

In [113]:
isbn_mean_count.columns = ['mean_rating', 'count_rating']

In [114]:
isbn_mean_count

,mean_rating,count_rating
isbn,,
0000913154,8.0,1
000104687X,6.0,1
0001047213,8.0,1
0001047973,9.0,2
000104799X,6.0,1
...,...,...
B0001FZGBC,7.0,1
B0001FZGPI,6.0,1
B0001FZGRQ,9.0,1


In [130]:
# 평균 평점이 3점 이하이고, 3개 이상의 count인 책들 집계
bad_books = isbn_mean_count[(isbn_mean_count['mean_rating'] <= 3.0) & (isbn_mean_count['count_rating'] >= 3.0)]

In [116]:
bad_books.sample(1)

,mean_rating,count_rating
isbn,,
0340632755,3.0,3


In [117]:
len(bad_books)

142

In [14]:
# bad books로 집계된 책들이 train에 얼마나 있을까?


In [131]:
# 평균 평점이 8점 이상이고, 3개 이상의 count인 책들 집계
good_books = isbn_mean_count[(isbn_mean_count['mean_rating'] >= 8.0) & (isbn_mean_count['count_rating'] >= 3.0)]

In [119]:
len(good_books)

6338

In [120]:
good_books.sample(1)

,mean_rating,count_rating
isbn,,
0618249060,9.5,4


In [132]:
# 평균 평점이 n점 이하이고, 3개 이상의 count인 책들 집계
bad_books_dict = dict()
for n in range(1,5):
    bad_books_dict[n] = isbn_mean_count[(isbn_mean_count['mean_rating'] <= n) & (isbn_mean_count['count_rating'] >= 3.0)]
    print(f"평균 평점이 {n}점 이하이고, 3개 이상의 count인 책들 집계")
    print(len(bad_books_dict[n]))
    print((bad_books_dict[n]['count_rating']).sum())
    if n == 1:
        print(f"구간별 count: {(bad_books_dict[n]['count_rating']).sum()}")
        tmp = (bad_books_dict[n]['count_rating']).sum()
    else:
        print(f"구간별 count: {(bad_books_dict[n]['count_rating']).sum() - tmp}")
        tmp = (bad_books_dict[n]['count_rating']).sum()

평균 평점이 1점 이하이고, 3개 이상의 count인 책들 집계
4
12
구간별 count: 12
평균 평점이 2점 이하이고, 3개 이상의 count인 책들 집계
29
98
구간별 count: 86
평균 평점이 3점 이하이고, 3개 이상의 count인 책들 집계
142
464
구간별 count: 366
평균 평점이 4점 이하이고, 3개 이상의 count인 책들 집계
568
2516
구간별 count: 2052


In [133]:
# 평균 평점이 n점 이상이고, 3개 이상의 count인 책들 집계
good_books_dict = dict()
for n in range(10, 5, -1):
    good_books_dict[n] = isbn_mean_count[(isbn_mean_count['mean_rating'] >= n) & (isbn_mean_count['count_rating'] >= 3.0)]
    print(f"평균 평점이 {n}점 이상이고, 3개 이상의 count인 책들 집계")
    print(len(good_books_dict[n]))
    print((good_books_dict[n]['count_rating']).sum())
    if n == 10:
        print(f"구간별 count: {(good_books_dict[n]['count_rating']).sum()}")
        tmp = (good_books_dict[n]['count_rating']).sum()
    else:
        print(f"구간별 count: {(good_books_dict[n]['count_rating']).sum() - tmp}")
        tmp = (good_books_dict[n]['count_rating']).sum()

평균 평점이 10점 이상이고, 3개 이상의 count인 책들 집계
54
169
구간별 count: 169
평균 평점이 9점 이상이고, 3개 이상의 count인 책들 집계
1381
5884
구간별 count: 5715
평균 평점이 8점 이상이고, 3개 이상의 count인 책들 집계
6338
39908
구간별 count: 34024
평균 평점이 7점 이상이고, 3개 이상의 count인 책들 집계
13511
115508
구간별 count: 75600
평균 평점이 6점 이상이고, 3개 이상의 count인 책들 집계
18696
161268
구간별 count: 45760


In [26]:
train_user_ids = train['user_id'].unique()
test_user_ids = test['user_id'].unique()
tmp = set(train_user_ids)
print(train['user_id'].nunique())
print(test['user_id'].nunique())

59803
26167


In [27]:
cnt = 0
for test_id in test_user_ids:
    if test_id in tmp:
        cnt += 1

In [28]:
cnt

17901

In [29]:
17901/26167

0.6841059349562426

In [123]:
ensemble = pd.read_csv('./submit/ensemble_2.1643-DCN_msk-DCN_msk2-sw-0.7-0.15-0.15.csv')

In [124]:
(bad_books_dict[1])

,mean_rating,count_rating
isbn,,
0373262957,1.0,3
0515090492,1.0,3
0670884839,1.0,3
3596136199,1.0,3


In [38]:
good_books_dict[10].index

Index(['0060099089', '0060245875', '0060973757', '0062517317', '006440546X',
       '0064405478', '0307131084', '0310925681', '0345332075', '037570440X',
       '0385480334', '0394832922', '039483609X', '0395082560', '0395873460',
       '0439042445', '0440415993', '0440512158', '0441005055', '0451133196',
       '0451527348', '0486270661', '0517124858', '0553568817', '0670825212',
       '0671041657', '0671600613', '0671698419', '0679726470', '0689823517',
       '0689851863', '0723236186', '0771099843', '0786807091', '0786867086',
       '0809232669', '0810919397', '0811803201', '0811826848', '0842377506',
       '0875421261', '087701759X', '0877792208', '0882435205', '0891343377',
       '0911662251', '0962713708', '0966067630', '1400061598', '1563054671',
       '1567184464', '1568650108', '1856975770', '1893956288'],
      dtype='object', name='isbn')

In [125]:
# ensemble[ensemble['isbn'] == good_books_dict[10].index]
for isbn in good_books_dict[9].index:
    try:
        print(train.loc[train['isbn'] == isbn]['rating'])
        # print(ensemble.loc[ensemble['isbn'] == isbn]['rating'])
        # print(good_books_dict[10].loc[isbn]['mean_rating'])
    except:
        continue

254874    10
254875     7
254876    10
254877     9
Name: rating, dtype: int64
238867    10
238868     9
238869    10
238870     8
Name: rating, dtype: int64
195560     8
195561     9
195562    10
195563     8
195564    10
Name: rating, dtype: int64
103612     9
103613    10
103614     9
103615     9
103616    10
103617     9
Name: rating, dtype: int64
153732     9
153733     7
153734    10
153735    10
Name: rating, dtype: int64
113399    10
113400     9
113401    10
113402     8
113403    10
Name: rating, dtype: int64
55336     9
55337    10
55338    10
Name: rating, dtype: int64
150958     9
150959     8
150960    10
Name: rating, dtype: int64
215560     9
215561    10
215562    10
215563    10
Name: rating, dtype: int64
232166    10
232167    10
232168     9
232169    10
232170    10
Name: rating, dtype: int64
225150    10
225151     7
225152    10
Name: rating, dtype: int64
229174    10
229175     9
229176     9
Name: rating, dtype: int64
255313    10
255314    10
255315     7
Nam

In [126]:
# ensemble[ensemble['isbn'] == good_books_dict[10].index]
for isbn in good_books_dict[9].index:
    try:
        # print(train.loc[train['isbn'] == isbn]['rating'])
        print(ensemble.loc[ensemble['isbn'] == isbn]['rating'])
        # print(good_books_dict[10].loc[isbn]['mean_rating'])
    except:
        continue

# 평균적으로 9점을 받는 책을, 모델이 6점 이하로 예측을 했으면, 잘못된 예측이라서
# 그거를 8점으로 보정을 하기로 생각했어

63719    6.726313
Name: rating, dtype: float64
59717    7.1517
Name: rating, dtype: float64
48890    7.802311
48891    7.312150
Name: rating, dtype: float64
25903    7.165217
25904    6.626159
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
28350    7.797006
28351    8.499670
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
37740    8.369596
Name: rating, dtype: float64
53890    8.302726
Name: rating, dtype: float64
58042    7.664063
Name: rating, dtype: float64
56288    7.243182
Name: rating, dtype: float64
57294    7.559975
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
39199    7.56019
Name: rating, dtype: float64
57977    8.237078
Name: rating, dtype: float64
15523    7.622123
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
37263    7.057527
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
62491    6.611026
Name: rating, dtype: float6

In [69]:
# ensemble[ensemble['isbn'] == good_books_dict[10].index]
for isbn in bad_books_dict[2].index:
    try:
        print(train.loc[train['isbn'] == isbn]['rating'])
        # print(ensemble.loc[ensemble['isbn'] == isbn]['rating'])
        # print(bad_books_dict[10].loc[isbn]['mean_rating'])
    except:
        continue

214658    1
214659    1
214660    4
Name: rating, dtype: int64
185745    3
185746    1
185747    2
Name: rating, dtype: int64
167232    1
167233    2
167234    3
Name: rating, dtype: int64
203657    1
203658    1
203659    1
Name: rating, dtype: int64
196216    1
196217    2
196218    3
Name: rating, dtype: int64
232084    1
232085    2
232086    1
Name: rating, dtype: int64
134416    2
134417    1
134418    3
Name: rating, dtype: int64
265581    2
265582    3
265583    1
Name: rating, dtype: int64
165758    1
165759    4
165760    1
Name: rating, dtype: int64
138508    1
138509    1
138510    1
Name: rating, dtype: int64
117951    4
117952    1
117953    1
Name: rating, dtype: int64
140477    1
140478    2
140479    3
Name: rating, dtype: int64
78936    1
78937    1
78938    1
Name: rating, dtype: int64
140809    1
140810    1
140811    3
Name: rating, dtype: int64
141751    1
141752    2
141753    2
Name: rating, dtype: int64
244024    5
244025    1
244026    1
244027    1
Name: rati

In [70]:
# ensemble[ensemble['isbn'] == good_books_dict[10].index]
for isbn in bad_books_dict[2].index:
    try:
        # print(train.loc[train['isbn'] == isbn]['rating'])
        print(ensemble.loc[ensemble['isbn'] == isbn]['rating'])
        # print(bad_books_dict[10].loc[isbn]['mean_rating'])
    except:
        continue

53665    6.085989
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
41440    6.767423
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
29488    6.44655
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
35438    6.435248
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
Series([], Name: rating, dtype: float64)
18868    6.932414
Name: rating, dtype: float64
Series([], Name: rating, dtype: float64)
3844    5.967571
Name: rating, dtype: float64
48578    4.930643
Name: rating, dtype: float64
43352    6.075048
Name: rating, dtype: float64
47067    6.

In [85]:
# ensemble[ensemble['isbn'] == good_books_dict[10].index]
# 평균 평점이 9점 and 평가 개수가 3개 이상인 isbn
cnt = 0
for isbn in good_books_dict[9].index:
    try:
        # print(train.loc[train['isbn'] == isbn]['rating'])
        # print(ensemble.loc[(ensemble['isbn'] == isbn) & (ensemble['rating'] <= 6)])
        ensemble.loc[(ensemble['isbn'] == isbn) & (ensemble['rating'] <= 6), 'rating'] = 8
        # print(ensemble[(ensemble['isbn'] == isbn) & (ensemble['rating'] <= 10)]['ratings'])
        # print(ensemble.loc[(ensemble['isbn'] == isbn)]['ratings'])
        # print(ensemble.loc[ensemble['isbn'] == isbn]['rating'])
        # print(good_books_dict[10].loc[isbn]['mean_rating'])
        cnt += 1
    except:
        continue
print(cnt)

Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
Empty DataFrame
Columns: [user_id, isbn, rating]
Index: 

In [134]:
# ensemble[ensemble['isbn'] == good_books_dict[10].index]
cnt = 0
for isbn in good_books_dict[10].index:
    try:
        # print(train.loc[train['isbn'] == isbn]['rating'])
        # print(ensemble.loc[(ensemble['isbn'] == isbn) & (ensemble['rating'] <= 6)])
        ensemble.loc[(ensemble['isbn'] == isbn) & (ensemble['rating'] <= 7), 'rating'] = 8
        # print(ensemble[(ensemble['isbn'] == isbn) & (ensemble['rating'] <= 10)]['ratings'])
        # print(ensemble.loc[(ensemble['isbn'] == isbn)]['ratings'])
        # print(ensemble.loc[ensemble['isbn'] == isbn]['rating'])
        # print(good_books_dict[10].loc[isbn]['mean_rating'])
        cnt += 1
    except:
        continue
print(cnt)

54


In [88]:
# ensemble[ensemble['isbn'] == good_books_dict[10].index]
cnt = 0
for isbn in bad_books_dict[2].index:
    try:
        # print(train.loc[train['isbn'] == isbn]['rating'])
        # print(ensemble.loc[(ensemble['isbn'] == isbn) & (ensemble['rating'] >= 2)])
        ensemble.loc[(ensemble['isbn'] == isbn) & (ensemble['rating'] >= 2), 'rating'] = 3
        # print(ensemble[(ensemble['isbn'] == isbn) & (ensemble['rating'] <= 10)]['ratings'])
        # print(ensemble.loc[(ensemble['isbn'] == isbn)]['ratings'])
        # print(ensemble.loc[ensemble['isbn'] == isbn]['rating'])
        # print(good_books_dict[10].loc[isbn]['mean_rating'])
        cnt += 1
    except:
        continue
print(cnt)

In [135]:
# ensemble[ensemble['isbn'] == good_books_dict[10].index]
cnt = 0
for isbn in bad_books_dict[1].index:
    try:
        # print(train.loc[train['isbn'] == isbn]['rating'])
        # print(ensemble.loc[(ensemble['isbn'] == isbn) & (ensemble['rating'] >= 2)])
        ensemble.loc[(ensemble['isbn'] == isbn) & (ensemble['rating'] >= 2), 'rating'] = 3
        # print(ensemble[(ensemble['isbn'] == isbn) & (ensemble['rating'] <= 10)]['ratings'])
        # print(ensemble.loc[(ensemble['isbn'] == isbn)]['ratings'])
        # print(ensemble.loc[ensemble['isbn'] == isbn]['rating'])
        # print(good_books_dict[10].loc[isbn]['mean_rating'])
        cnt += 1
    except:
        continue
print(cnt)

4


In [100]:
def special_round(x:float, a:float):
    if x < 1:
        return 1
    else:
        criteria = round(x)
        return criteria if criteria - a <= x <= criteria + a else x

In [102]:
ensemble_1 = ensemble.copy()
# ensemble_2 = ensemble.copy()

In [103]:
ensemble_1['rating'] = ensemble_1['rating'].apply(lambda x: special_round(x, 0.1))
ensemble_1.to_csv('./submit/ensemble_0.1.csv')
# ensemble_2['rating'] = ensemble_2['rating'].apply(lambda x: special_round(x, 0.2))
# ensemble_2.to_csv('./submit/ensemble_0.2.csv')
# 6.8 7.2 -> 7

In [ ]:
# x: 1.1
# a: 0.1
# round(x) -> 1 -> criteria = round(x) -> 1 - 0.1 = 0.9, 1 + 0.1 = 1.1
# 0.9 <= x <= 1.1 -> round(x)

# x: 1.5
# a: 0.1
# round(x) -> 2 -> criteria = round(x) -> 2 - 0.1 = 1.9, 2 + 0.1 = 2.1
# 1.9 <= x <= 2.1 -> x